# Overview of Tables Generated 

The tables that are created for this demo 

- **Loan Acquisition Table** – Stores key origination attributes for newly acquired loans, enabling analysis of lender trends, borrower risk profiles, and product mix.  
- **Loan Performance Table** – Tracks monthly payment behavior and status for each loan to support monitoring of credit performance, delinquency, and prepayment.  
- **Borrower Demographics Table** – Captures borrower demographic attributes used for housing goals, fair lending analysis, and regulatory reporting.  
- **Property Characteristics Table** – Holds core collateral and property attributes used for underwriting, geographic segmentation, and valuation analytics.  
- **Loan Servicing Table** – Records which servicer handles each loan and any servicing transfers, supporting servicer performance and compliance monitoring.  
- **Mortgage Note Table** – Contains terms of the mortgage note (rate, term, type) used for compliance checks and analysis of product structures like fixed vs ARM.  
- **Loss Severity Table** – Stores realized loss amounts and related details after liquidation events to support loss modeling and credit risk analytics.  
- **Loan Application Table** – Contains application‑stage borrower and loan information used for eligibility checks, fraud analytics, and process optimization.  
- **Loan Repayment History Table** – Provides a month‑by‑month record of billed and paid amounts to support prepayment, delinquency, and collections analytics.  
- **Housing Goals Compliance Table** – Flags loans that meet specific affordable housing and minority lending criteria for tracking against regulatory goals.  
- **Uniform Closing Data Table** – Standardizes key closing‑time data elements needed for regulatory compliance, audits, and payment validation.  
- **Property Appraisal Table** – Stores appraisal values and related details used to assess collateral strength, valuation risk, and pricing decisions.  
- **MBS Pooling Table** – Describes how loans are grouped into MBS pools to support pool‑level performance tracking and investor reporting.  
- **Credit History Table** – Summarizes borrower credit characteristics at origination for use in risk scoring, underwriting, and portfolio analytics.  
- **Unit Rental Table** – Contains unit‑level rent and occupancy details for properties, supporting income verification, rental risk assessment, and housing supply analysis.


In [0]:
CATALOG = 'main_jcg'
SCHEMA = 'fnma'

## Loan Acquisition Table

**Description**: Contains details of newly acquired loans.


**Use case**: Track originator trends, credit risk profiling, eligibility analysis.


**Columns**: Loan ID, Origination Date, Originator, Property Address, Borrower Income, LTV Ratio, Loan Amount, Loan Purpose, Product Type, First-Time Homebuyer Indicator, APR Spread, Borrower Race, Borrower Ethnicity, Borrower Gender

In [0]:
import pandas as pd
import random
import numpy as np

num_rows = 700

originators = ["Bank of America", "Wells Fargo", "Chase", "Quicken Loans", "CitiMortgage", "Guaranteed Rate", "Rocket Mortgage"]
loan_purposes = ["Purchase", "Refinance", "Home Equity"]
product_types = ["Fixed Rate", "Adjustable Rate", "Interest Only", "Balloon"]
races = ["White", "Black", "Asian", "Native American", "Pacific Islander", "Other", "Missing"]
ethnicities = ["Hispanic or Latino", "Not Hispanic or Latino", "Missing"]
genders = ["Male", "Female", "Other", "Missing"]
street_names = ["Main St", "Park Ave", "Broadway", "Elm St", "Oak Dr", "Maple Rd", "Sunset Blvd"]
cities = ["Atlanta", "Chicago", "Houston", "Los Angeles", "New York", "Miami", "Seattle"]
states = ["GA", "IL", "TX", "CA", "NY", "FL", "WA"]

random.seed(42)
np.random.seed(42)

def random_address():
    num = random.randint(100, 9999)
    street = random.choice(street_names)
    city = random.choice(cities)
    state = random.choice(states)
    zip_code = random.randint(10000, 99999)
    return f"{num} {street}, {city}, {state} {zip_code}"

data = []
for i in range(num_rows):
    loan_id = f"LN{100000 + i}"
    orig_date = pd.Timestamp('2020-01-01') + pd.to_timedelta(random.randint(0,1800), unit='D')
    originator = random.choice(originators)
    prop_addr = random_address()
    borrower_income = int(np.random.normal(85000, 30000))
    ltv_ratio = round(np.random.uniform(50, 98), 2)
    loan_amount = int(np.random.normal(350000, 100000))
    loan_purpose = random.choice(loan_purposes)
    product_type = random.choice(product_types)
    first_time = random.choice([True, False])
    apr_spread = round(np.random.normal(2.5, 0.75), 3)
    race = random.choice(races)
    ethnicity = random.choice(ethnicities)
    gender = random.choice(genders)
    
    data.append([
        loan_id, orig_date.strftime('%Y-%m-%d'), originator, prop_addr, borrower_income, ltv_ratio, loan_amount,
        loan_purpose, product_type, first_time, apr_spread, race, ethnicity, gender
    ])

columns = [
    "Loan_ID","Origination_Date", "Originator", "Property_Address", "Borrower_Income", "LTV_Ratio", "Loan_Amount",
    "Loan_Purpose", "Product_Type", "First_Time_Homebuyer", "APR_Spread", "Borrower_Race", "Borrower_Ethnicity", "Borrower_Gender"
]

loan_acq_df = pd.DataFrame(data, columns=columns)
# loan_acq_df.to_csv('loan_acquisition_table_sample.csv', index=False)
print(loan_acq_df.head())


    Loan_ID Origination_Date  ...      Borrower_Ethnicity Borrower_Gender
0  LN100000       2023-08-02  ...  Not Hispanic or Latino            Male
1  LN100001       2020-03-02  ...  Not Hispanic or Latino           Other
2  LN100002       2024-07-15  ...  Not Hispanic or Latino            Male
3  LN100003       2020-07-08  ...      Hispanic or Latino         Missing
4  LN100004       2020-06-10  ...                 Missing          Female

[5 rows x 14 columns]


In [0]:
loan_acq_df_sparkles = spark.createDataFrame(loan_acq_df)
loan_acq_df_sparkles.write.format("delta").mode('overwrite').saveAsTable(f'{CATALOG}.{SCHEMA}.loan_acquisition')

## Loan Performance Table
**Description**: Monthly updates about loan payment status and performance metrics.

**Use case**: Monitor credit performance, default risk, prepayment trends.

**Columns**: Loan ID, Performance Month, Outstanding Balance, Payment Received, Delinquency Status, Days Delinquent, Current Interest Rate, Property Disposition Status

In [0]:
random.seed(42)
np.random.seed(42)

num_rows = random.randint(700, 1000)

months = pd.date_range(start='2020-01-01', end='2025-11-01', freq='MS').strftime('%Y-%m')
loan_ids = [f"LN{100000 + i}" for i in range(num_rows)]
delinquency_status_options = ["Current", "30 Days", "60 Days", "90+ Days", "Default", "Foreclosed", "Paid Off"]
disposition_status_options = ["Active", "Foreclosed", "Paid Off", "REO Sold", "Short Sale", "No Disposition"]

data = []
for i in range(num_rows):
    loan_id = random.choice(loan_ids)
    perf_month = random.choice(months)
    out_balance = int(abs(np.random.normal(250000, 90000)))
    payment_received = int(np.random.uniform(0, 3000))
    delinq_status = random.choice(delinquency_status_options)
    days_delinq = 0 if delinq_status == "Current" else random.choice([30, 60, 90, 120, 150, 180, 365])
    curr_rate = round(np.random.uniform(2.5, 7.0), 3)
    disp_status = random.choice(disposition_status_options)
    data.append([
        loan_id, perf_month, out_balance, payment_received, delinq_status, days_delinq, curr_rate, disp_status
    ])

columns = [
    "Loan_ID", "Performance_Month", "Outstanding_Balance", "Payment_Received",
    "Delinquency_Status", "Days_Delinquent", "Current_Interest_Rate", "Property_Disposition_Status"
]

loan_perf_df = pd.DataFrame(data, columns=columns)
loan_perf_df_sparkles = spark.createDataFrame(loan_perf_df)
loan_perf_df_sparkles.write.format("delta").mode('overwrite').saveAsTable(f'{CATALOG}.{SCHEMA}.loan_performance')

## Borrower Demographics Table
**Description**: Demographic profile of borrowers for housing goals and compliance.


**Use case**: Affordable housing tracking, regulator reporting, diversity analysis.


**Columns**: Borrower ID, Race, Ethnicity, Gender, Date of Birth, Income, Number of Borrowers

In [0]:
random.seed(42)
np.random.seed(42)
num_rows = random.randint(700, 1000)

races = ["White", "Black", "Asian", "Native American", "Pacific Islander", "Other", "Missing"]
ethnicities = ["Hispanic or Latino", "Not Hispanic or Latino", "Missing"]
genders = ["Male", "Female", "Other", "Missing"]

# Random DOB generator
start_date = pd.Timestamp('1940-01-01')
end_date = pd.Timestamp('2004-01-01')

def random_dob():
    return (start_date + pd.to_timedelta(random.randint(0, (end_date-start_date).days), unit='D')).strftime('%Y-%m-%d')

data = []
for i in range(num_rows):
    borrower_id = f"BW{200000 + i}"
    race = random.choice(races)
    ethnicity = random.choice(ethnicities)
    gender = random.choice(genders)
    dob = random_dob()
    income = int(np.random.normal(85000, 30000))
    num_borrowers = random.choice([1, 2, 3])
    data.append([
        borrower_id, race, ethnicity, gender, dob, income, num_borrowers
    ])

columns = ["Borrower_ID", "Race", "Ethnicity", "Gender", "Date_of_Birth", "Income", "Number_of_Borrowers"]
borr_demo_df = pd.DataFrame(data, columns=columns)
borr_demo_df_sparkles = spark.createDataFrame(borr_demo_df)
borr_demo_df_sparkles.write.format("delta").mode('overwrite').saveAsTable(f'{CATALOG}.{SCHEMA}.borrower_demographics')

## Property Characteristics Table
**Description**: Key property attributes related to collateral.


**Use case**: Underwriting, risk segmentation, valuation analytics.


**Columns**: Property ID, Loan ID, Property Address, Dwelling Type, Number of Units, Number of Bedrooms, Geographic Census Tract, Metropolitan Division Code.


In [0]:
random.seed(45)
np.random.seed(45)
num_rows = random.randint(700, 1000)

dwelling_types = ["Single Family", "Condo", "Townhouse", "2-4 Unit", "Manufactured Home", "Co-op", "Other"]
census_tracts = [f"{random.randint(100000,999999)}" for _ in range(50)]
metro_div_codes = [f"MD{random.randint(100,999)}" for _ in range(20)]
street_names = ["Main St", "Park Ave", "Broadway", "Elm St", "Oak Dr", "Maple Rd", "Sunset Blvd"]
cities = ["Atlanta", "Chicago", "Houston", "Los Angeles", "New York", "Miami", "Seattle"]
states = ["GA", "IL", "TX", "CA", "NY", "FL", "WA"]

def random_address():
    num = random.randint(100,9999)
    street = random.choice(street_names)
    city = random.choice(cities)
    state = random.choice(states)
    zip_code = random.randint(10000,99999)
    return f"{num} {street}, {city}, {state} {zip_code}"

data = []
for i in range(num_rows):
    property_id = f"PR{300000 + i}"
    loan_id = f"LN{100000 + random.randint(0, 999)}"
    prop_addr = random_address()
    dwelling_type = random.choice(dwelling_types)
    num_units = random.choice([1, 2, 3, 4])
    num_bedrooms = random.choice([1, 2, 3, 4, 5, 6])
    census_tract = random.choice(census_tracts)
    metro_div_code = random.choice(metro_div_codes)
    data.append([
        property_id, loan_id, prop_addr, dwelling_type, num_units, num_bedrooms, census_tract, metro_div_code
    ])

columns = ["Property_ID", "Loan_ID", "Property_Address", "Dwelling_Type", "Number_of_Units", "Number_of_Bedrooms", "Geographic_Census_Tract", "Metropolitan_Division_Code"]
property_df = pd.DataFrame(data, columns=columns)
property_df_sparkles = spark.createDataFrame(property_df)
property_df_sparkles.write.format("delta").mode('overwrite').saveAsTable(f'{CATALOG}.{SCHEMA}.property_characteristics')

## Loan Servicing Table
**Description**: Information about loan servicers and servicing transfers.


**Use case**: Servicing performance analysis, transfer history, compliance.


**Columns**: Loan ID, Servicer Name, Servicer Transfer Date, Previous Servicer, Servicer Ratings.

In [0]:
random.seed(62)
np.random.seed(62)
num_rows = random.randint(700, 1000)

servicers = ["Wells Fargo", "Mr. Cooper", "Chase", "Flagstar Bank", "U.S. Bank", "Quicken Loans", "PHH Mortgage"]
servicer_ratings = ["Excellent", "Good", "Average", "Below Average", "Poor"]

start_date = pd.Timestamp('2019-01-01')
end_date = pd.Timestamp('2025-11-01')

def random_transfer_date():
    return (start_date + pd.to_timedelta(random.randint(0, (end_date-start_date).days), unit='D')).strftime('%Y-%m-%d')

data = []
for i in range(num_rows):
    loan_id = f"LN{100000 + random.randint(0, 999)}"
    servicer_name = random.choice(servicers)
    transfer_date = random_transfer_date()
    prev_servicer = random.choice([s for s in servicers if s != servicer_name])
    servicer_rating = random.choice(servicer_ratings)
    data.append([
        loan_id, servicer_name, transfer_date, prev_servicer, servicer_rating
    ])

columns = ["loan_id", "servicer_name", "servicer_transfer_date", "previous_servicer", "servicer_ratings"]
servicing_df = pd.DataFrame(data, columns=columns)

# Saving as a delta table
servicing_df_sparkles = spark.createDataFrame(servicing_df)
servicing_df_sparkles.write.format("delta").mode('overwrite').saveAsTable(f'{CATALOG}.{SCHEMA}.loan_servicing_table')

## Mortgage Note Table
**Description**: Details of mortgage notes for compliance and underwriting.


**Use case**: Regulatory compliance, ARM/FRM analysis.


**Columns**: Loan ID, Mortgage Note Date, Note Rate, Term, Amortization Type, HOEPA Status, Loan Type

In [0]:
random.seed(3)
np.random.seed(3)
num_rows = random.randint(700, 1000)

terms = [15, 20, 30, 40]
amort_types = ["Fixed", "Adjustable", "Interest_Only", "Balloon"]
hoepa_statuses = ["Yes", "No"]
loan_types = ["Conventional", "FHA", "VA", "USDA"]

start_date = pd.Timestamp('2018-01-01')
end_date = pd.Timestamp('2025-11-01')

def random_note_date():
    return (start_date + pd.to_timedelta(random.randint(0, (end_date-start_date).days), unit='D')).strftime('%Y-%m-%d')

data = []
for i in range(num_rows):
    loan_id = f"LN{100000 + random.randint(0, 999)}"
    note_date = random_note_date()
    note_rate = round(np.random.uniform(2.5, 7.0), 3)
    term = random.choice(terms)
    amort_type = random.choice(amort_types)
    hoepa_status = random.choice(hoepa_statuses)
    loan_type = random.choice(loan_types)
    data.append([
        loan_id, note_date, note_rate, term, amort_type, hoepa_status, loan_type
    ])

columns = [
    "Loan_Id", "Mortgage_Note_Date", "Note_Rate", "Term", "Amortization_Type", "HOEPA_Status", "Loan_Type"
]
note_df = pd.DataFrame(data, columns=columns)

# Saving as a delta table
note_df_sparkles = spark.createDataFrame(note_df)
note_df_sparkles.write.format("delta").mode('overwrite').saveAsTable(f'{CATALOG}.{SCHEMA}.loan_note_table')

## Loss Severity Table
**Description**: Losses recorded after foreclosures or short sales.


**Use case**: Loss modeling, severity analytics, credit risk reporting.


**Columns**: Loan ID, Loss Amount, Disposition Date, Cause of Loss, Recoveries

In [0]:
random.seed(10)
np.random.seed(10)
num_rows = random.randint(700, 1000)

cause_of_loss_options = ["Foreclosure", "Short_Sale", "Deed_in_Lieu", "REO_Sale"]

start_date = pd.Timestamp('2018-01-01')
end_date = pd.Timestamp('2025-11-01')

def random_disposition_date():
    return (start_date + pd.to_timedelta(random.randint(0, (end_date-start_date).days), unit='D')).strftime('%Y-%m-%d')

data = []
for i in range(num_rows):
    loan_id = f"LN{100000 + random.randint(0, 999)}"
    loss_amount = round(abs(np.random.normal(50000, 40000)), 2)
    disposition_date = random_disposition_date()
    cause_of_loss = random.choice(cause_of_loss_options)
    recoveries = round(np.random.uniform(0, loss_amount * 0.8), 2)
    data.append([
        loan_id, loss_amount, disposition_date, cause_of_loss, recoveries
    ])

columns = ["Loan_Id", "Loss_Amount", "Disposition_Date", "Cause_of_Loss", "Recoveries"]
loss_df = pd.DataFrame(data, columns=columns)

# Saving as a delta table
loss_df_sparkles = spark.createDataFrame(loss_df)
loss_df_sparkles.write.format("delta").mode('overwrite').saveAsTable(f'{CATALOG}.{SCHEMA}.loss_severity_table')

## Loan Application Table
**Description**: Captures fields from consumer loan applications.


**Use case**: Fraud analytics, eligibility, process improvement.


**Columns**: Application ID, Submission Date, Borrower ID, Property ID, Employment History, Total Debt, Monthly Income, Loan Purpose

In [0]:
random.seed(15)
np.random.seed(15)
num_rows = random.randint(700, 1000)

loan_purposes = ["Purchase", "Refinance", "Home_Equity"]
employment_hist_options = ["Employed_5+_Years", "Employed_2-5_Years", "Employed_Less_Than_2_Years", "Unemployed", "Retired", "Self_Employed"]

start_date = pd.Timestamp('2018-01-01')
end_date = pd.Timestamp('2025-11-01')

def random_submission_date():
    return (start_date + pd.to_timedelta(random.randint(0, (end_date-start_date).days), unit='D')).strftime('%Y-%m-%d')

data = []
for i in range(num_rows):
    application_id = f"APP{400000 + i}"
    submission_date = random_submission_date()
    borrower_id = f"BW{200000 + random.randint(0, 999)}"
    property_id = f"PR{300000 + random.randint(0, 999)}"
    employment_history = random.choice(employment_hist_options)
    total_debt = int(abs(np.random.normal(100000, 40000)))
    monthly_income = int(np.random.normal(7000, 2500))
    loan_purpose = random.choice(loan_purposes)
    data.append([
        application_id, submission_date, borrower_id, property_id, employment_history, total_debt, monthly_income, loan_purpose
    ])

columns = ["Application_Id", "Submission_Date", "Borrower_Id", "Property_Id", "Employment_History", "Total_Debt", "Monthly_Income", "Loan_Purpose"]
application_df = pd.DataFrame(data, columns=columns) 

# Saving as a delta table
application_df_sparkles = spark.createDataFrame(application_df)
application_df_sparkles.write.format("delta").mode('overwrite').saveAsTable(f'{CATALOG}.{SCHEMA}.application_table')

## Loan Repayment History Table
**Description**: Historical monthly payment records for each loan.


**Use case**: Prepayment modeling, delinquency analysis.


**Columns**: Loan ID, Payment Date, Amount Due, Amount Paid, Payment Status, Days Past Due.

In [0]:
random.seed(35)
np.random.seed(35)
num_rows = random.randint(700, 1000)

payment_status_options = ["Current", "Late", "Missed", "Partial", "Paid_Off"]

start_date = pd.Timestamp('2020-01-01')
end_date = pd.Timestamp('2025-11-01')

def random_payment_date():
    return (start_date + pd.to_timedelta(random.randint(0, (end_date-start_date).days), unit='D')).strftime('%Y-%m-%d')

data = []
for i in range(num_rows):
    loan_id = f"LN{100000 + random.randint(0, 999)}"
    payment_date = random_payment_date()
    amount_due = int(abs(np.random.normal(2000, 800)))
    amount_paid = int(max(0, np.random.normal(amount_due, 500)))
    pay_status = random.choice(payment_status_options)
    days_past_due = 0 if pay_status == "Current" else random.choice([5, 15, 30, 45, 60])
    data.append([
        loan_id, payment_date, amount_due, amount_paid, pay_status, days_past_due
    ])

columns = ["Loan_Id", "Payment_Date", "Amount_Due", "Amount_Paid", "Payment_Status", "Days_Past_Due"]
repayment_df = pd.DataFrame(data, columns=columns)

# Saving as a Delta Table
repayment_df_sparkles = spark.createDataFrame(repayment_df)
repayment_df_sparkles.write.format("delta").mode('overwrite').saveAsTable(f'{CATALOG}.{SCHEMA}.repayment_table')

## Housing Goals Compliance Table
**Description**: Tracks loans meeting affordable housing and minority lending goals.


**Use case**: Goal compliance, regulatory reporting, program tracking.


**Columns**: Loan ID, Housing Goal Met Indicator, Income Category, Census Tract Minority Category

In [0]:
random.seed(6899)
np.random.seed(6899)
num_rows = random.randint(700, 1000)

goal_met_options = ["Yes", "No"]
income_categories = ["Low", "Moderate", "Middle", "Upper"]
minority_categories = ["Majority_Minority", "Non_Minority", "Intermediate", "Unknown"]

data = []
for i in range(num_rows):
    loan_id = f"LN{100000 + random.randint(0, 999)}"
    goal_met = random.choice(goal_met_options)
    income_category = random.choice(income_categories)
    minority_category = random.choice(minority_categories)
    data.append([
        loan_id, goal_met, income_category, minority_category
    ])

columns = ["Loan_Id", "Housing_Goal_Met_Indicator", "Income_Category", "Census_Tract_Minority_Category"]
housing_goal_df = pd.DataFrame(data, columns=columns)

# Saving as a Delta Table
housing_goal_df_sparkles = spark.createDataFrame(housing_goal_df)
housing_goal_df_sparkles.write.format("delta").mode('overwrite').saveAsTable(f'{CATALOG}.{SCHEMA}.housing_goal_table')

##Uniform Closing Data Table
**Description**: Standardized closing data required by CFPB.


**Use case**: Regulatory compliance, audit support, payment validation.


**Columns**: Loan ID, Closing Disclosure Date, Fees Paid, Closing Agent, Final Loan Amount, Final APR

In [0]:
num_rows = random.randint(700, 1000)

closing_agents = ["First American", "Stewart Title", "Fidelity National", "Old Republic", "Chicago Title", "HomeServices"]

start_date = pd.Timestamp('2018-01-01')
end_date = pd.Timestamp('2025-11-01')

def random_closing_date():
    return (start_date + pd.to_timedelta(random.randint(0, (end_date-start_date).days), unit='D')).strftime('%Y-%m-%d')

data = []
for i in range(num_rows):
    loan_id = f"LN{100000 + random.randint(0, 999)}"
    closing_date = random_closing_date()
    fees_paid = round(abs(np.random.normal(6000, 2000)), 2)
    closing_agent = random.choice(closing_agents)
    final_loan_amt = int(abs(np.random.normal(350000, 90000)))
    final_apr = round(np.random.uniform(2.5, 7.5), 3)
    data.append([
        loan_id, closing_date, fees_paid, closing_agent, final_loan_amt, final_apr
    ])

columns = ["Loan_Id", "Closing_Disclosure_Date", "Fees_Paid", "Closing_Agent", "Final_Loan_Amount", "Final_APR"]
closing_df = pd.DataFrame(data, columns=columns)

# Saving as a Delta Table
closing_df_sparkles = spark.createDataFrame(closing_df)
closing_df_sparkles.write.format("delta").mode('overwrite').saveAsTable(f'{CATALOG}.{SCHEMA}.uniform_closing_table')


## Property Appraisal Table
**Description**: Appraisal data captured for loan collateral and risk.


**Use case**: Underwriting, collateral risk segmentation, valuation trends.


**Columns**: Property ID, Appraisal Date, Value, Appraiser ID, Appraisal Type.

In [0]:
appraisal_types = ["Full_Appraisal", "Drive-By", "Automated", "Desktop", "Re-Inspection"]

start_date = pd.Timestamp('2018-01-01')
end_date = pd.Timestamp('2025-11-01')

def random_appraisal_date():
    return (start_date + pd.to_timedelta(random.randint(0, (end_date-start_date).days), unit='D')).strftime('%Y-%m-%d')

data = []
for i in range(num_rows):
    property_id = f"PR{300000 + random.randint(0, 999)}"
    appraisal_date = random_appraisal_date()
    value = int(np.random.normal(400000, 120000))
    appraiser_id = f"AP{50000 + random.randint(0, 9999)}"
    appraisal_type = random.choice(appraisal_types)
    data.append([
        property_id, appraisal_date, value, appraiser_id, appraisal_type
    ])

columns = ["Property_Id", "Appraisal_Date", "Value", "Appraiser_Id", "Appraisal_Type"]
appraisal_df = pd.DataFrame(data, columns=columns)

# Saving as a Delta Table
appraisal_df_sparkles = spark.createDataFrame(appraisal_df)
appraisal_df_sparkles.write.format("delta").mode('overwrite').saveAsTable(f'{CATALOG}.{SCHEMA}.property_appraisal_table')

## MBS Pooling Table
**Description**: Allocation of loans to mortgage-backed securities (MBS).


**Use case**: Pool tracking, investor reporting, security structure analytics.


**Columns**: Pool ID, Security Issue Date, Component Loan IDs, Aggregate UPB (Unpaid Principal Balance), Pool Credit Score.

In [0]:
num_rows = random.randint(700, 1000)

start_date = pd.Timestamp('2018-01-01')
end_date = pd.Timestamp('2025-11-01')

pool_ids = [f"POOL{90000 + i}" for i in range(num_rows)]
loan_universe = [f"LN{100000 + i}" for i in range(2000)]
component_sizes = [random.randint(3, 10) for _ in range(num_rows)]

data = []
for i in range(num_rows):
    pool_id = pool_ids[i]
    issue_date = (start_date + pd.to_timedelta(random.randint(0, (end_date - start_date).days), unit='D')).strftime('%Y-%m-%d')
    component_loan_ids = ",".join(random.sample(loan_universe, component_sizes[i]))
    agg_upb = int(abs(np.random.normal(1700000, 450000)))  # aggregate UPB across loans in pool
    pool_credit_score = int(np.random.normal(715, 40))
    data.append([
        pool_id, issue_date, component_loan_ids, agg_upb, pool_credit_score
    ])

columns = ["Pool_Id", "Security_Issue_Date", "Component_Loan_Ids", "Aggregate_UPB", "Pool_Credit_Score"]
mbs_df = pd.DataFrame(data, columns=columns)

# Saving as a Delta Table
mbs_df_sparkles = spark.createDataFrame(mbs_df)
mbs_df_sparkles.write.format("delta").mode('overwrite').saveAsTable(f'{CATALOG}.{SCHEMA}.mbs_table')

## Credit History Table
**Description**: Borrower credit profile at time of loan origination.


**Use case:** Pre-qualification, risk scoring, trend analysis.


**Columns**: Borrower ID, Number/Age of Credit Accounts, Payment History, Credit Inquiries, Debt-to-Income Ratio

In [0]:
num_rows = random.randint(700, 1000)

payment_hist_options = ["Clean", "Late_30", "Late_60", "Late_90", "Defaulted", "Mixed"]

data = []
for i in range(num_rows):
    borrower_id = f"BW{200000 + random.randint(0, 999)}"
    num_credit_accts = random.randint(1, 12)
    age_credit_accts = random.randint(1, 25)  # in years
    payment_history = random.choice(payment_hist_options)
    credit_inquiries = random.randint(0, 8)
    dti_ratio = round(np.random.uniform(15, 55), 2)  # percent
    data.append([
        borrower_id, num_credit_accts, age_credit_accts,
        payment_history, credit_inquiries, dti_ratio
    ])

columns = ["Borrower_Id", "Number_Credit_Accounts", "Age_Credit_Accounts", "Payment_History", "Credit_Inquiries", "Debt_to_Income_Ratio"]
credit_df = pd.DataFrame(data, columns=columns)

# Saving as a Delta Table
credit_df_sparkles = spark.createDataFrame(credit_df)
credit_df_sparkles.write.format("delta").mode('overwrite').saveAsTable(f'{CATALOG}.{SCHEMA}.credit_history')

## Unit Rental Table
**Description**: Rental income and rent-related data for multifamily single-family units.


**Use case**: Income verification, risk assessment, housing supply analysis.


**Columns**: Property ID, Unit ID, Unit Rent, Number of Bedrooms, Eligible Rental Indicator

In [0]:
num_rows = random.randint(700, 1000)

eligible_rental_options = ["Yes", "No"]
unit_ids = [f"UNIT{1000 + i}" for i in range(num_rows)]

data = []
for i in range(num_rows):
    property_id = f"PR{300000 + random.randint(0, 999)}"
    unit_id = unit_ids[i]
    unit_rent = int(abs(np.random.normal(2000, 650)))
    num_bedrooms = random.choice([1, 2, 3, 4, 5])
    eligible_rental_indicator = random.choice(eligible_rental_options)
    data.append([
        property_id, unit_id, unit_rent, num_bedrooms, eligible_rental_indicator
    ])

columns = ["Property_Id", "Unit_Id", "Unit_Rent", "Number_of_Bedrooms", "Eligible_Rental_Indicator"]
unit_rental_df = pd.DataFrame(data, columns=columns)

# Saving as a Delta Table
unit_rental_df_sparkles = spark.createDataFrame(unit_rental_df)
unit_rental_df_sparkles.write.format("delta").mode('overwrite').saveAsTable(f'{CATALOG}.{SCHEMA}.unit_rentals')

# Data Products 

## Acquisition & Origination Analytics

- **Purpose:** Track trends in loan acquisition, borrower profiles, and application processes.
- **Tables:** Loan Acquisition, Loan Application, Borrower Demographics, Credit History.
- **Ideal Customers:** Fannie Mae acquisition teams, lenders, risk modelers, business analysts.
- **Uses:** Identify emerging origination patterns, gauge credit risk at entry point, understand borrower diversity and affordability.

## Loan Performance & Servicing

- **Purpose:** Monitor monthly payment, delinquency, and servicing transfers across the portfolio.
- **Tables:** Loan Performance, Repayment History, Loan Servicing.
- **Ideal Customers:** Portfolio/regulatory risk managers, servicing analytics teams, secondary market investors.
- **Uses:** Evaluate ongoing loan and servicer performance, track delinquencies, inform default analytics.

## Collateral & Property Intelligence

- **Purpose:** Centralize and analyze property details and valuations for risk and compliance.
- **Tables:** Property Characteristics, Property Appraisal, Unit Rental.
- **Ideal Customers:** Underwriting teams, real estate investors, collateral risk, and affordable housing program managers.
- **Uses:** Validate property value, assess geographic and affordable housing patterns, analyze rental income streams.

## Compliance & Regulatory Reporting

- **Purpose:** Fulfill mandates for housing goals, fair lending, and consumer protection.
- **Tables:** Housing Goals Compliance, Uniform Closing Data, Borrower Demographics.
- **Ideal Customers:** Regulatory compliance, legal teams, advocacy organizations, and program managers.
- **Uses:** Track progress on minority lending, affordable housing, and CFPB closing rules.

## Loss and Credit Risk Management

- **Purpose:** Quantify realized losses and inform risk models.
- **Tables:** Loss Severity, Loan Performance, Credit History.
- **Ideal Customers:** Credit risk model development teams, capital markets, secondary investors.
- **Uses:** Analyze loss severity and causes, model default and prepayment risk, price credit risk transfer products.

## Investor and Security Allocation

- **Purpose:** Support pooling, allocation, and reporting for MBS and credit risk transfer.
- **Tables:** MBS Pooling, Loan Acquisition, Loan Performance.
- **Ideal Customers:** Capital markets analytics, investor relations, financial engineers.
- **Uses:** Structure securities, track pool-level performance, provide timely investor reporting.

## Fraud Detection and Process Optimization (Cross-Functional)

- **Purpose:** Leverage application, origination, servicing, and transaction data for anomaly detection and workflow improvement.
- **Tables:** Loan Application, Loan Servicing, Uniform Closing Data, Credit History.
- **Ideal Customers:** Enterprise innovation or fraud analytics teams, internal audit, data science partners.
- **Uses:** Detect fraud, optimize application/servicing processes, improve accuracy and compliance.


In [0]:
# Data Product Tag list
{
  "acquisition_and_origination_analytics": "Acquisition & Origination Analytics",
  "loan_performance_and_servicing": "Loan Performance & Servicing",
  "collateral_and_property_intelligence": "Collateral & Property Intelligence",
  "compliance_and_regulatory_reporting": "Compliance & Regulatory Reporting",
  "loss_and_credit_risk_management": "Loss and Credit Risk Management",
  "investor_and_security_allocation": "Investor and Security Allocation",
  "fraud_detection_and_process_optimization": "Fraud Detection and Process Optimization"
}


In [0]:
# By tags for each table
table_tags = {
  "loan_acquisition": "acquisition_and_origination_analytics,investor_and_security_allocation",
  "application_table": "acquisition_and_origination_analytics,fraud_detection_and_process_optimization",
  "borrower_demographics": "acquisition_and_origination_analytics,compliance_and_regulatory_reporting",
  "credit_history": "acquisition_and_origination_analytics,loss_and_credit_risk_management,fraud_detection_and_process_optimization",
  "loan_performance": "loan_performance_and_servicing,loss_and_credit_risk_management,investor_and_security_allocation",
  "repayment_table": "loan_performance_and_servicing",
  "loan_servicing_table": "loan_performance_and_servicing,fraud_detection_and_process_optimization",
  "property_characteristics": "collateral_and_property_intelligence",
  "property_appraisal_table": "collateral_and_property_intelligence",
  "unit_rentals": "collateral_and_property_intelligence",
  "housing_goal_table": "compliance_and_regulatory_reporting",
  "uniform_closing_table": "compliance_and_regulatory_reporting,fraud_detection_and_process_optimization",
  "loss_severity_table": "loss_and_credit_risk_management",
  "mbs_table": "investor_and_security_allocation",
}


In [0]:
#Product as a key, List of tables as the values in a list
products_and_tables = {
  "acquisition_and_origination_analytics": [
    "loan_acquisition",
    "application_table",
    "borrower_demographics",
    "credit_history"
  ],
  "investor_and_security_allocation": [
    "loan_acquisition",
    "loan_performance",
    "mbs_table"
  ],
  "fraud_detection_and_process_optimization": [
    "application_table",
    "credit_history",
    "loan_servicing_table",
    "uniform_closing_table"
  ],
  "compliance_and_regulatory_reporting": [
    "borrower_demographics",
    "housing_goal_table",
    "uniform_closing_table"
  ],
  "loss_and_credit_risk_management": [
    "credit_history",
    "loan_performance",
    "loss_severity_table"
  ],
  "loan_performance_and_servicing": [
    "loan_performance",
    "repayment_table",
    "loan_servicing_table"
  ],
  "collateral_and_property_intelligence": [
    "property_characteristics",
    "property_appraisal_table",
    "unit_rentals"
  ]
}


In [0]:
CATALOG = "fnma_product_catalog_jcg"
SCHEMA = "default"

rows = [
    (
        product,
        table,
        f"{CATALOG}.{SCHEMA}.{table}"
    )
    for product, tables in products_and_tables.items()
    for table in tables
]

df = spark.createDataFrame(
    rows,
    ["product", "table_name", "full_table_name"]
)

display(df)

df.write.mode("overwrite").saveAsTable("fnma_product_catalog_jcg.default.tables_and_products")

product,table_name,full_table_name
acquisition_and_origination_analytics,loan_acquisition,fnma_product_catalog_jcg.default.loan_acquisition
acquisition_and_origination_analytics,application_table,fnma_product_catalog_jcg.default.application_table
acquisition_and_origination_analytics,borrower_demographics,fnma_product_catalog_jcg.default.borrower_demographics
acquisition_and_origination_analytics,credit_history,fnma_product_catalog_jcg.default.credit_history
investor_and_security_allocation,loan_acquisition,fnma_product_catalog_jcg.default.loan_acquisition
investor_and_security_allocation,loan_performance,fnma_product_catalog_jcg.default.loan_performance
investor_and_security_allocation,mbs_table,fnma_product_catalog_jcg.default.mbs_table
fraud_detection_and_process_optimization,application_table,fnma_product_catalog_jcg.default.application_table
fraud_detection_and_process_optimization,credit_history,fnma_product_catalog_jcg.default.credit_history
fraud_detection_and_process_optimization,loan_servicing_table,fnma_product_catalog_jcg.default.loan_servicing_table


12/10 Update: 
Table for product, table, and description

In [0]:
%sql

SELECT
  t.TABLE_NAME,
  t.COMMENT AS Description
FROM main_jcg.INFORMATION_SCHEMA.TABLES t
WHERE t.TABLE_SCHEMA = 'fnma' 

TABLE_NAME,Description
catalog_metadata,"The table contains information about various assets within the system. It includes details such as the type of asset, its name, and associated tags. This data can be used for asset management, tracking, and categorization purposes, helping teams understand the relationships and attributes of different assets."
property_appraisal_table,"The table contains data related to property appraisals. It includes information such as the property identifier, appraisal date, appraised value, appraiser details, and the type of appraisal conducted. This data can be used for analyzing property values over time, assessing appraisal trends, and evaluating the performance of appraisers."
uniform_closing_table,"The table contains data related to loan closings. It includes information such as the loan ID, closing disclosure date, fees paid, closing agent, final loan amount, and final annual percentage rate (APR). This data can be used for analyzing loan closing processes, understanding fee structures, and evaluating the performance of closing agents."
products_and_tags,"The table contains a list of tags along with their descriptions. This data can be used for categorizing and organizing information, making it easier to search and filter content based on specific tags. It is useful for projects that require tagging systems, such as content management or data classification."
property_characteristics,"The table contains data related to properties and their associated loans. It includes details such as property addresses, dwelling types, and the number of units and bedrooms. This information can be used for analyzing property characteristics, understanding loan distributions across different types of properties, and assessing geographic trends in property financing."
catalog_search_index,Managed Vector Index with Delta Sync
loan_note_table,"The table contains data related to mortgage loans. It includes details such as loan identifiers, mortgage note dates, interest rates, loan terms, amortization types, HOEPA status, and loan types. This information can be used for analyzing mortgage products, assessing loan performance, and understanding lending trends."
catalog_search_index_writeback_table,null
mbs_table,"The table contains data related to loan pools and their associated security issues. It includes information such as the pool identifier, the date of the security issue, loan IDs within the pool, the aggregate unpaid balance (UPB), and the credit score of the pool. This data can be used for analyzing loan performance, assessing risk factors, and understanding trends in loan pools."
product_search_index_writeback_table,null


In [0]:
%sql
SELECT
  tp.product,
  t.TABLE_NAME,
  t.COMMENT AS Description
FROM main_jcg.INFORMATION_SCHEMA.TABLES t
LEFT JOIN fnma_product_catalog_jcg.default.tables_and_products tp
  ON t.TABLE_NAME = tp.table_name
WHERE t.TABLE_SCHEMA = 'fnma' and tp.product is not null

product,TABLE_NAME,Description
collateral_and_property_intelligence,property_appraisal_table,"The table contains data related to property appraisals. It includes information such as the property identifier, appraisal date, appraised value, appraiser details, and the type of appraisal conducted. This data can be used for analyzing property values over time, assessing appraisal trends, and evaluating the performance of appraisers."
compliance_and_regulatory_reporting,uniform_closing_table,"The table contains data related to loan closings. It includes information such as the loan ID, closing disclosure date, fees paid, closing agent, final loan amount, and final annual percentage rate (APR). This data can be used for analyzing loan closing processes, understanding fee structures, and evaluating the performance of closing agents."
fraud_detection_and_process_optimization,uniform_closing_table,"The table contains data related to loan closings. It includes information such as the loan ID, closing disclosure date, fees paid, closing agent, final loan amount, and final annual percentage rate (APR). This data can be used for analyzing loan closing processes, understanding fee structures, and evaluating the performance of closing agents."
collateral_and_property_intelligence,property_characteristics,"The table contains data related to properties and their associated loans. It includes details such as property addresses, dwelling types, and the number of units and bedrooms. This information can be used for analyzing property characteristics, understanding loan distributions across different types of properties, and assessing geographic trends in property financing."
investor_and_security_allocation,mbs_table,"The table contains data related to loan pools and their associated security issues. It includes information such as the pool identifier, the date of the security issue, loan IDs within the pool, the aggregate unpaid balance (UPB), and the credit score of the pool. This data can be used for analyzing loan performance, assessing risk factors, and understanding trends in loan pools."
loan_performance_and_servicing,repayment_table,"The table contains information related to loan payments. It includes details such as the loan identifier, payment dates, amounts due and paid, payment status, and the number of days past due. This data can be used to track loan repayment behaviors, assess payment performance, and identify trends in overdue payments."
compliance_and_regulatory_reporting,borrower_demographics,"The table contains demographic and financial information about borrowers. It includes details such as race, ethnicity, gender, date of birth, income, and the number of borrowers associated with each record. This data can be used for analyzing borrower profiles, understanding lending patterns, and assessing the impact of demographic factors on borrowing behavior."
acquisition_and_origination_analytics,borrower_demographics,"The table contains demographic and financial information about borrowers. It includes details such as race, ethnicity, gender, date of birth, income, and the number of borrowers associated with each record. This data can be used for analyzing borrower profiles, understanding lending patterns, and assessing the impact of demographic factors on borrowing behavior."
loan_performance_and_servicing,loan_servicing_table,"The table contains information about loan servicers and their performance. It includes details such as the loan ID, the name of the current servicer, the date of any transfer of servicing, the previous servicer, and ratings for the servicer. This data can be used to analyze servicer performance over time, track changes in loan servicing, and assess customer satisfaction with different servicers."
fraud_detection_and_process_optimization,loan_servicing_table,"The table contains information about loan servicers and their performance. It includes details such as the loan ID, the name of the current servicer, the

In [0]:
%sql
CREATE or REPLACE TABLE main_jcg.FNMA.products_tables_descriptions as
(SELECT
  tp.product,
  collect_list(
    named_struct(
      'table_name', t.table_name,
      'description', t.comment
    )
  ) AS tables_and_descriptions
FROM main_jcg.information_schema.tables t
LEFT JOIN fnma_product_catalog_jcg.default.tables_and_products tp
  ON t.table_name = tp.table_name
WHERE t.table_schema = 'fnma'
  AND tp.product IS NOT NULL
GROUP BY tp.product)

num_affected_rows,num_inserted_rows


In [0]:
rows = spark.sql("""
  SELECT
    tp.table_name,
    tp.product_schema,
    t.comment AS table_comment
  FROM main_jcg.fnma.table_products tp
  JOIN main_jcg.information_schema.tables t
    ON t.table_schema = 'fnma'
   AND t.table_name  = tp.table_name
""").collect()

for r in rows:
    table_name     = r["table_name"]
    product_schema = r["product_schema"]          # e.g. acquisition_and_origination_analytics
    base_comment   = r["table_comment"] or ""
    view_comment   = f"View for {product_schema} over {table_name}. {base_comment}".strip()
    view_comment   = view_comment.replace("'", "''")  # escape quotes

    spark.sql(f"""
      COMMENT ON VIEW main_jcg.fnma_data_catalog.{product_schema}.{table_name}
      IS '{view_comment}'
    """)


In [0]:

#For a single table
table_name = "loan_acquisition"

sql_command = f"""
ALTER TABLE {CATALOG}.{SCHEMA}.{table_name}
SET TAGS ('product' = 'acquisition_and_origination_analytics,investor_and_security_allocation')
"""

spark.sql(sql_command)



DataFrame[]

In [0]:
for table, tags in table_tags.items():
    sql_command = f"ALTER TABLE {CATALOG}.{SCHEMA}.{table} SET TAGS ('product' = '{tags}')"
    spark.sql(sql_command)


# Product Table with Tags and Descriptions

In [0]:
products_and_tags = [
  {
    "tag": "acquisition_and_origination_analytics",
    "description": "Purpose: Track trends in loan acquisition, borrower profiles, and application processes. Uses: Identify emerging origination patterns, gauge credit risk at entry point, understand borrower diversity and affordability."
  },
  {
    "tag": "loan_performance_and_servicing",
    "description": "Purpose: Monitor monthly payment, delinquency, and servicing transfers across the portfolio. Uses: Evaluate ongoing loan and servicer performance, track delinquencies, inform default analytics."
  },
  {
    "tag": "collateral_and_property_intelligence",
    "description": "Purpose: Centralize and analyze property details and valuations for risk and compliance. Uses: Validate property value, assess geographic and affordable housing patterns, analyze rental income streams."
  },
  {
    "tag": "compliance_and_regulatory_reporting",
    "description": "Purpose: Fulfill mandates for housing goals, fair lending, and consumer protection. Uses: Track progress on minority lending, affordable housing, and CFPB closing rules."
  },
  {
    "tag": "loss_and_credit_risk_management",
    "description": "Purpose: Quantify realized losses and inform risk models. Uses: Analyze loss severity and causes, model default and prepayment risk, price credit risk transfer products."
  },
  {
    "tag": "investor_and_security_allocation",
    "description": "Purpose: Support pooling, allocation, and reporting for MBS and credit risk transfer. Uses: Structure securities, track pool-level performance, provide timely investor reporting."
  },
  {
    "tag": "fraud_detection_and_process_optimization",
    "description": "Purpose: Leverage application, origination, servicing, and transaction data for anomaly detection and workflow improvement. Uses: Detect fraud, optimize application/servicing processes, improve accuracy and compliance."
  }
]



In [0]:
products_and_tags_df = spark.createDataFrame(products_and_tags)
products_and_tags_df = products_and_tags_df.select("tag", "description")
display(products_and_tags_df)

# Creating a products and tags table in UC
products_and_tags_df.write.mode("overwrite").saveAsTable(f"{CATALOG}.{SCHEMA}.products_and_tags")

tag,description
acquisition_and_origination_analytics,"Purpose: Track trends in loan acquisition, borrower profiles, and application processes. Uses: Identify emerging origination patterns, gauge credit risk at entry point, understand borrower diversity and affordability."
loan_performance_and_servicing,"Purpose: Monitor monthly payment, delinquency, and servicing transfers across the portfolio. Uses: Evaluate ongoing loan and servicer performance, track delinquencies, inform default analytics."
collateral_and_property_intelligence,"Purpose: Centralize and analyze property details and valuations for risk and compliance. Uses: Validate property value, assess geographic and affordable housing patterns, analyze rental income streams."
compliance_and_regulatory_reporting,"Purpose: Fulfill mandates for housing goals, fair lending, and consumer protection. Uses: Track progress on minority lending, affordable housing, and CFPB closing rules."
loss_and_credit_risk_management,"Purpose: Quantify realized losses and inform risk models. Uses: Analyze loss severity and causes, model default and prepayment risk, price credit risk transfer products."
investor_and_security_allocation,"Purpose: Support pooling, allocation, and reporting for MBS and credit risk transfer. Uses: Structure securities, track pool-level performance, provide timely investor reporting."
fraud_detection_and_process_optimization,"Purpose: Leverage application, origination, servicing, and transaction data for anomaly detection and workflow improvement. Uses: Detect fraud, optimize application/servicing processes, improve accuracy and compliance."


In [0]:
%sql
-- So the vector search index can stay up to date
ALTER TABLE main_jcg.FNMA.products_and_tags SET TBLPROPERTIES (delta.enableChangeDataFeed = true);

Create a vector search index on the product and tags table
